In [37]:
import numpy as np
import pandas as pd
from pysr import PySRRegressor
import matplotlib.pyplot as plt
import math

In [38]:
def generate_x(num_points, num_vars, variables, equation_row):
    X = np.zeros((num_points, num_vars))

    for i in range(1, num_vars):
        var_name = equation_row[f'v{i}_name']
        var_low = equation_row[f'v{i}_low']
        var_high = equation_row[f'v{i}_high']
        variables[var_name] = np.random.uniform(var_low, var_high, num_points)
        X[:, i-1] = variables[var_name]

    np_funcs = {func: getattr(np, func) for func in dir(np) if callable(getattr(np, func))}
    np_consts = {const: getattr(np, const) for const in dir(np) if not callable(getattr(np, const))}
    
    np_funcs = {
        'sin': np.sin, 'cos': np.cos, 'tan': np.tan,
        'exp': np.exp, 'log': np.log, 'ln': np.log,  # ln as alias for log
        'sqrt': np.sqrt, 'abs': np.abs,
        'pi': np.pi, 'e': np.e,
        'arcsin': np.arcsin, 'arccos': np.arccos, 'arctan': np.arctan,
        'tanh': np.tanh,
    }

    # Combine all namespaces
    eval_namespace = {**np_funcs, **np_consts, **variables}

    return X, variables, eval_namespace

In [39]:
def load_equations(filename):
    return pd.read_csv(filename)

def generate_data(equation_row, num_points=100):
    formula = equation_row['Formula']
    num_vars = int(equation_row['# variables'])

    variables = {}
    
    try:
        X, variables, eval_namespace = generate_x(num_points, num_vars,variables, equation_row)
        y = eval(formula, eval_namespace)
    except Exception as e:
        X, variables, eval_namespace = generate_x(num_points, num_vars + 1, variables, equation_row)
        y = eval(formula, eval_namespace)

    print(variables.keys())
    return X, y

def run_regression(equation_row):
    X, y = generate_data(equation_row)

    model = PySRRegressor(
        niterations=100,
        binary_operators=["+", "*", "/", "-"],
        unary_operators=["exp", "sqrt"],
        populations=20,
        ncycles_per_iteration=500,
        maxsize=20,
        verbosity=0,
        temp_equation_file=True,
    )

    model.fit(X, y)

    best_equation = model.sympy()
    print(f"Best equation: {best_equation}")

    y_pred = model.predict(X)

    # calculate MSE
    mse = np.mean((y - y_pred) ** 2)
    print(f"MSE: {mse}")
    return mse

In [40]:
import warnings
warnings.filterwarnings("ignore")

In [41]:
equations = load_equations('FeynmanEquations.csv')
run_regression(equations.iloc[80])

dict_keys(['n_rho', 'mom', 'B', 'kb', 'T'])
Best equation: x0*(x1 - sqrt(x3*x4/x2)) + x0
MSE: 0.4749685941841237


np.float64(0.4749685941841237)

In [42]:
mse_val = np.zeros(100)
equations = load_equations('FeynmanEquations.csv')
for i in range (0, 100):
    print(i)
    mse_val[i] = run_regression(equations.iloc[i])

print(f"Mean MSE: {np.mean(mse_val)}")

0
dict_keys(['theta'])
Best equation: 0.3989423/sqrt(exp(x0**2))
MSE: 3.0436772707041675e-17
1
dict_keys(['sigma', 'theta'])
Best equation: 0.10873686*(-2.2759373)/x0 + 1.1796887*0.65652645/(x0*exp(x1*0.47027144/x0))
MSE: 1.2380603075487244e-05
2
dict_keys(['sigma', 'theta', 'theta1'])
Best equation: -x3 + 0.40459424/(x0 + 0.64080584/(x0/((-x1 + x2)*(-x1 + x2)) - 1*0.100057624))
MSE: 1.338116072117309e-05
3
dict_keys(['x1', 'x2', 'y1', 'y2'])
Best equation: sqrt((-x2 + x3)**2 + exp((x0 - 1.8423289)/x1))
MSE: 0.22870739598479697
4
dict_keys(['m1', 'm2', 'G', 'x1', 'x2', 'y1', 'y2', 'z1', 'z2'])
Best equation: x0*x1*x2*sqrt(x8)*(x4 + x6)/(x3*x5*x7)
MSE: 0.00035511302349568966
5
dict_keys(['m_0', 'v', 'c'])
Best equation: x0*x1/(x2*(((-x1*x1 + x1 + x2 + x2)/x1))) + x0
MSE: 7.809551450294396e-05
6
dict_keys(['x1', 'x2', 'x3', 'y1', 'y2', 'y3'])
Best equation: x0*x3 + x1*x4 + x2*x5
MSE: 0.0
7
dict_keys(['mu', 'Nn'])
Best equation: x0*x1
MSE: 0.0
8
dict_keys(['q1', 'q2', 'epsilon', 'r'])
Bes